ДЗ по итогам лекции 16.

"Использовать полученные знания в части обучения моделей для обучения и подбора параметров в задачах из предыдущих ДЗ на выбор (мфо, задача с тем возьмёт ли человек кредит или нет)
В частности использовать:

GaussianNB

LogisticRegression 

Linear + PolynomialFeatures

RandomForestClassifier 

При обучении использовать make_pipeline

GridSearchCV для поиска параметров 

Посчитать метрики (f1 + точность + AUC). "


Анализ задания и план работы. В части подготовки данных повторяем шаги из предыдущих работ.
Следует загрузить файл данных credit_train.csv, представленный в формате csv, в датафрейм модуля pandas.
*) Далее следует провести поиск пропусков и удалить строки с пропусками.
*) Кроме того, предварительный анализ файла данных показывает, что колонка living region содержит много дублей типа "московская обл" и "обл московская" или "чеченская республика" и "чеченская респ". Придется поработать с данными этой колонки, чтобы привести систему обозначений регионов к однозначной. Поскольку регионов слишком много, что затруднит анализ корреляции данных, то мы приведем всю систему к федеральным округам и двум столицам - Москва и Санкт-Петербург.
*) Далее для оценки выбросов числовых данных следует провести оценку матожидания и дисперсии каждого столбца данных, представленного в численном виде. Все значения, отстоящие более чем на 3 выборочных стандартных отклонения, удаляем из массива данных (точнее, удаляем из выборки все объекты, для которых хотя бы в одном столбце имеются выбросы).
*) С помощью функций подключенной библиотек оценим нормальность распределений данных
*) Числовые колонки данных масштабируем, поделив на максимальный элемент каждой колонки


Далее, после подготовки данных, выполняем анализ.

Загружаем данные.
Выводим для ознакомления заголовочную строку и первые пять строк. 

In [85]:
%reset
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

data_table = pd.read_csv('credit_train.csv', sep = ';', decimal = ',', encoding = 'windows-1251')
print(f'Rows in the initial table: {len(data_table)}') 
print(f'Initial column number: {data_table.shape[1]}')
print(data_table.head())

Rows in the initial table: 170746
Initial column number: 15
   client_id gender   age marital_status job_position  credit_sum  \
0          1      M   NaN            NaN          UMN    59998.00   
1          2      F   NaN            MAR          UMN    10889.00   
2          3      M  32.0            MAR          SPC    10728.00   
3          4      F  27.0            NaN          SPC    12009.09   
4          5      M  45.0            NaN          SPC         NaN   

   credit_month tariff_id  score_shk education        living_region  \
0            10       1.6        NaN       GRD   КРАСНОДАРСКИЙ КРАЙ   
1             6       1.1        NaN       NaN               МОСКВА   
2            12       1.1        NaN       NaN      ОБЛ САРАТОВСКАЯ   
3            12       1.1        NaN       NaN    ОБЛ ВОЛГОГРАДСКАЯ   
4            10       1.1   0.421385       SCH  ЧЕЛЯБИНСКАЯ ОБЛАСТЬ   

   monthly_income  credit_count  overdue_credit_count  open_account_flg  
0         30000.0       

Видим сразу много NaN. 
Это процедура загрузки так заменила пропуски.
Такие строки подлежат устранению.
Удаляем строки c NaN (то есть с отсутствовавшими изначально значениями, которые при загрузке с помощью функции read_csv превратились в NaN) c помощью dropna:

In [86]:
data_table.dropna(inplace=True) # inplace means we change data_table itself, not create new modified one
print(data_table.head())
print(f'Размер таблицы {data_table.shape}')

    client_id gender   age marital_status job_position  credit_sum  \
7           8      F  26.0            UNM          SPC     47878.0   
9          10      F  32.0            UNM          SPC     26268.0   
10         11      M  26.0            MAR          SPC     16793.0   
12         13      M  37.0            MAR          SPC     42289.0   
13         14      M  43.0            MAR          INP     57567.0   

    credit_month tariff_id  score_shk education        living_region  \
7             10       1.1   0.512525       GRD       МОСКОВСКАЯ ОБЛ   
9             10       1.1   0.465026       GRD  КРАЙ СТАВРОПОЛЬСКИЙ   
10            14       1.0   0.445430       SCH      САНКТ-ПЕТЕРБУРГ   
12            10       1.6   0.691609       SCH    ОБЛ АРХАНГЕЛЬСКАЯ   
13            10       1.1   0.341164       GRD  ХАНТЫ-МАНСИЙСКИЙ АО   

    monthly_income  credit_count  overdue_credit_count  open_account_flg  
7          60000.0           3.0                   0.0                 

Теперь следует обработать колонку living region. 
Поскольку регионов слишком много, работа сразу со всеми даст громоздкие результаты, анализ которых затруднен.
Поэтому мы объединим все регионы по федеральным округам плюс отдельно укажем Москву и Петербург. 
Получим ровно десять living regions:
1. Центральный ФО
2. Северо-Западный ФО
3. Южный ФО
4. Северо-Кавказский ФО
5. Приволжский  ФО
6. Уральский ФО
7. Сибирский ФО
8. Дальневосточный ФО
9. Москва
10. Санкт-Петербург

С таким количеством регионов уже можно работать.
Попутно отметим, что в таблице два раза встречается регион "Россия". Его убираем.

In [87]:
# looking for the region 'Россия' and removing it:
indices = data_table.index[data_table['living_region'] == 'РОССИЯ']
for ind in indices:
    data_table = data_table.drop(index = ind)

# function returning the string replace_to if a data_cell contains a string from string_set
def replace_cell_value(data_cell, string_set, replace_to):
    for str in string_set:
        if str in data_cell:
            return replace_to            
    return data_cell # if no string in string_set is present in data_cell

# the regions of every federal district, keywords:
central_fo = ['БЕЛГОРОД', 'БРЯНСК', 'ВЛАДИМИР', 'ВОРОНЕЖ', 'ГУСЬ', 'ИВАНОВ', 'КАЛУГ', 'КАЛУЖ', 'КОСТРОМ', 'КУРСК', 'ЛИПЕЦК', 'МОСКОВСК', 'МЫТИЩ', 'ОРЕЛ', 'ОРЁЛ', 'ОРЛОВСК', 'РЯЗАН', 'СМОЛЕНСК', 'ТАМБОВ', 'ТВЕР', 'ТУЛА', 'ТУЛЬСК', 'ЯРОСЛАВ'] 
north_west_fo = ['КАРЕЛ','КОМИ','АРХАНГ','ВОЛОГ','КАЛИНИНГР','ЛЕНИНГР','МУРМАН','НОВГОРОД','ПСКОВ','НЕНЕЦК']
south_fo = ['АДЫГ','КАЛМЫК','КРЫМ','КРАСНОДАР','АСТРАХАН','ВОЛГОГРАД','РОСТОВ','СЕВАСТ']
caucas_fo =['ДАГЕСТ','ИНГУШ','КАБАРДИН','БАЛКАР','КАРАЧАЕВ','ЧЕРКЕС','ОСЕТИ','АЛАНИ','ЧЕЧЕН','ЧЕЧНЯ','СТАВРОПОЛ']
volga_fo = ['ПРИВОЛЖСК','БАШКИР','БАШКОР', 'ГОРЬКИЙ', 'ГОРЬКОВСК', 'МАРИЙ','МОРДОВ','ТАТАР','УДМУРТ','ЧУВАШ','ПЕРМ','КИРОВ','НИЖЕГОРОД','НИЖНИЙ','ОРЕНБУРГ','ПЕНЗ','САМАР','САРАТОВ','УЛЬЯН','СИМБИРС']
urals_fo =['ЕКАТЕРИНБ','СВЕРДЛОВ','КУРГАН','ТЮМЕН','ЧЕЛЯБ','ТАГИЛ','ХАНТЫ','МАНСИ','ЯМАЛ','НЕНЕЦ', '74']
siberia_fo = ['АЛТАЙ','ТЫВА','ТУВА','ТУВИН','ХАКАС','КРАСНОЯРСК','ИРКУТСК','КЕМЕРОВ','КУЗБАСС','НОВОСИБ','ОМСК', 'ЭВЕНК']
far_east_fo = ['ВЛАДИВОСТ','ХАБАРОВ','БУРЯТ','САХА','ЯКУТ','БАЙКАЛ','КАМЧАТ','ПРИМОРСК','АМУР','МАГАДАН','САХАЛИН','ЕВРЕЙ','ЧИТА', 'ЧИТИН', 'ЧУКОТ']
moskau = ['МОСКВ']
piter = ['ПЕТЕРБ', '98']
regions = [central_fo, north_west_fo, south_fo, caucas_fo, volga_fo, urals_fo, siberia_fo, far_east_fo, moskau, piter]

# regions to be put into dataset:
federal_districts = ['Центральный ФО', 'Северо-Западный ФО', 'Южный ФО', 'Северо-Кавказский ФО', 'Приволжский  ФО', 'Уральский ФО', 'Сибирский ФО', 'Дальневосточный ФО', 'Москва', 'Санкт-Петербург']

# Replacing the cells in living_region with federal districts
i = 0
for region in regions:
    data_table['living_region'] = data_table['living_region'].map(lambda x: replace_cell_value(x, region, federal_districts[i]))
    i += 1

# saving result for debugging purposes
# data_table.to_csv('tmp.csv', index = False, sep=';', decimal = ',', encoding = 'windows-1251') 
print(f'Now the table shape is: {data_table.shape}')
print(data_table.head())

Now the table shape is: (161329, 15)
    client_id gender   age marital_status job_position  credit_sum  \
7           8      F  26.0            UNM          SPC     47878.0   
9          10      F  32.0            UNM          SPC     26268.0   
10         11      M  26.0            MAR          SPC     16793.0   
12         13      M  37.0            MAR          SPC     42289.0   
13         14      M  43.0            MAR          INP     57567.0   

    credit_month tariff_id  score_shk education         living_region  \
7             10       1.1   0.512525       GRD        Центральный ФО   
9             10       1.1   0.465026       GRD  Северо-Кавказский ФО   
10            14       1.0   0.445430       SCH       Санкт-Петербург   
12            10       1.6   0.691609       SCH    Северо-Западный ФО   
13            10       1.1   0.341164       GRD          Уральский ФО   

    monthly_income  credit_count  overdue_credit_count  open_account_flg  
7          60000.0          

Все колонки делим на две категории - числовые и нечисловые. 
Числовые будут далее исследоваться на выбросы и затем проверяться на нормальность, затем нормироваться.
Нечисловые колонки кодируем энкодером OneHotEncoder.

In [88]:
# Column titles of categorical features
categorical_features_onehot = ['gender', 'marital_status', 'education', 'living_region'] # for encoding by onehot encoder
# Even though tariff_id consists from zeros and units, but is in fact non-numerical, since it is the tariff category
categorical_features_label = ['tariff_id', 'job_position']  # for encoding by label encoder
# Column titles of numerical features
numerical_features = ['age', 'credit_sum', 'credit_month', 'score_shk', 'monthly_income', 'credit_count', 'overdue_credit_count', 'open_account_flg']

# creating dataframe for encoded values. First including only numeric data there
data_table_encoded = data_table[numerical_features] # dataframe for encoded data set
data_table_encoded.reset_index(drop=True, inplace = True) # changing series indices to sequential incrementing values
# print(data_table_encoded.head()) 

# transform the categorical data using OneHotEncoder
encoder = OneHotEncoder() # init encoder
for feature in categorical_features_onehot:
    # print(feature)
    tmp_data = encoder.fit_transform(data_table[[feature]]) # coded data is sparse matrix
    tmp_data = tmp_data.todense() # to usual matrix 
    # print(tmp_data)
    # extract the feature names for the encoded columns
    tmp_titles = encoder.get_feature_names_out([feature])
    # print(tmp_titles)
    tmp_df = pd.DataFrame(tmp_data, columns = tmp_titles)    
    data_table_encoded = pd.concat([data_table_encoded, tmp_df], axis = 1, ignore_index = False, sort = False) # merging into the encoded dataframe     
    # print(f'Current table size: {data_table_encoded.shape}')

# transform other categorical data using LabelEncoder
encoder = LabelEncoder() # init encoder
for feature in categorical_features_label:
    # print(feature)    
    data_table_encoded[feature] = encoder.fit_transform(data_table[[feature]])    

print(f'New length of table: {len(data_table_encoded)}')
print(f'New column number: {data_table_encoded.shape[1]}')

# for debugging only:
# print(data_table_encoded.head())
# data_table_encoded.to_csv('tmp1.csv', sep = ';', decimal = '.', encoding = 'windows-1251')

New length of table: 161329
New column number: 32


c:\Users\varaksa_yua\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\varaksa_yua\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Рассчитываем выбросы и отсеиваем строки с выбросами (где значения отстоят больше чем на 3 стандартных отклонения от среднего).
Отметим, что из колонок типа 'overdue_credit_count', 'open_account_flg' отсеивать выбросы не следует. Дело в том, что, например, в колонке 'overdue_credit_count' большинство значений нулевые, и очень редко, например, число просроченных кредитов равно 1, 2 или 3. Тогда среднее значение будет околонулевым, дисперсия из-за подавляющего превосходства в числе нулевых значений тоже будет мала, и поэтому отсеивание выбросов по принципу, применимому для гауссова распределения, то есть отстоящих более чем на три сигмы от среднего значения, просто уберет из колонки все ненулевые значения и тем самым обнулит ценность самой колонки.
Поэтому для отсеивания выбросов отберем только колонки, где имеется множество различных значений, а именно: age, credit_sum, score_shk, monthly_income.

In [89]:
# calculating the means and standard deviations over some numerical columns
where_remove_outliers = ['age', 'credit_sum', 'score_shk', 'monthly_income']
# removing outliers (i.e. leaving only rows withot outliers)
# i = 1
for feature in where_remove_outliers:
    mean = data_table_encoded[feature].mean()
    std = data_table_encoded[feature].std()
    print(f'Feature: {feature}, mean: {mean:.1f}, std: {std:.1f}')
    # z_scores i.e. distance from the data value to mean, in std values
    z_scores = (data_table_encoded[feature] - mean) / std # absolute value of z score
    z_scores = z_scores.abs()
    # print(z_scores)
    # z_scores.to_csv('tmp_z'+ str(i) + '.csv', sep = ';', decimal = '.', encoding = 'windows-1251')
    # i += 1
    # if z_scores are > 3 the data is considered to be an outlier:
    data_table_encoded = data_table_encoded[z_scores <= 3] # leaving only items closer to the mean value (not farther than 3*sigma)
    # another variant:
    # mask = z_scores.abs().lt(3) # mask to remove all rows with z values >=3
    # data_table_encoded = data_table_encoded[mask] # leaving only items closer to the mean value (not farther than 3*sigma)
    print(f'Current table size: {data_table_encoded.size}')
 
data_table_encoded.reset_index(drop = True, inplace = True) # changing final series indices to incrementing values
data_table_encoded.to_csv('tmp_encoded.csv', sep = ';', decimal = '.', encoding = 'windows-1251')   
print(f'Now the table shape is: {data_table_encoded.shape}')  

Feature: age, mean: 37.0, std: 10.4
Current table size: 5151456
Feature: credit_sum, mean: 25942.8, std: 16194.6
Current table size: 5093632
Feature: score_shk, mean: 0.5, std: 0.1
Current table size: 5089888
Feature: monthly_income, mean: 40077.5, std: 24666.0
Current table size: 5021984
Now the table shape is: (156937, 32)


Далее следует масштабировать данные - то есть поделить числовые колонки на их максимумы.
В финале сохранить полученные предобработанные данные для дальнейшего использования в какой-либо модели.

In [90]:
for feature in numerical_features:
    data_table_encoded[feature] = data_table_encoded[feature] / data_table_encoded[feature].max()

data_table_encoded.to_csv('credit_train_preprocessed.csv', sep = ';', decimal = '.', encoding = 'windows-1251')  
print(data_table_encoded.shape) 

(156937, 32)


Учтем, что решаемая нами задача - определение по признакам кредитополучателей, получит ли кредит кредитополучатель, является задачей бинарной классификации. То есть мы должны по набору признаков кредитополучателя предсказать значение в колонке open_account_flg, показывающей, получил ли кредитополучатель кредит.
В задачах бинарной классификации бывает целесообразно сбалансировать обучающую выборку, то есть выравнять количество элементов обучающего набора, относящихся к каждому из двух классов кредитополучателей (получивших кредит, т.е. для которых open_account_flg == 1, и не получивших кредит, т.е. для которых open_account_flg == 0). 

In [91]:
tmp = data_table_encoded['open_account_flg'].value_counts() # extracting the number of each value in the column 'open_account_flg'
print(tmp)
# extracting all samples with 'open_account_flg'== 1:
samples_1 = data_table_encoded[data_table_encoded['open_account_flg'] == 1]
# extracting all samples with 'open_account_flg'== 0:
samples_0 = data_table_encoded[data_table_encoded['open_account_flg'] == 0]
# sampling a sub-sample from samples_0 with the size equal to the one of the  
samples_00 = samples_0.sample(n = tmp[1], random_state = 42) # 42 is the seed for random number generator

data_table_preprocessed = pd.concat([samples_1, samples_00]) # new data table with equal number of open_account_flg = 0 and = 1

open_account_flg
0.0    131084
1.0     25853
Name: count, dtype: int64


Проверяем размеры нового датасета:

In [92]:
print(data_table_preprocessed.shape)
print(data_table_preprocessed[data_table_preprocessed['open_account_flg'] == 0].shape)
print(data_table_preprocessed[data_table_preprocessed['open_account_flg'] == 1].shape)

(51706, 32)
(25853, 32)
(25853, 32)


Нужно разделить датасет на две выборки - обучающую и тестовую.
Но вначале нужно разделить таблицу данных на две: на набор колонок X, значения которых будут служить аргументом, подаваемым на вход модели, и колонку меток Y, на получение которой и натренировывается модель.

In [93]:
from sklearn.model_selection import train_test_split
X = data_table_preprocessed.drop(columns=['open_account_flg'])
Y = data_table_preprocessed['open_account_flg']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [94]:
# For debugging:
# X_train.to_csv('X_train.csv', sep = ';', decimal = '.', encoding = 'windows-1251')   

Проведем обучение классификатора GaussianNB (Gaussian Naive Bayesian). При этом в данном случае GridSearchCV для поиска параметров не используется, потому что для GaussianNB никакие параметры не подбираются. Как требуется по заданию, используем make_pipeline.
Для любого классификатора в sklearn используется похожая последовательность действий:
Создается экземпляр классификатора.
Проводится обучение методом fit.
Предсказываются значения меток с помощью обученного классификатора вызовом метода predict.

In [95]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB

# creating pipeline
pipe = make_pipeline(StandardScaler(), GaussianNB())

# training pipeline
pipe.fit(X_train, Y_train)

# predicting class labels
Y_pred = pipe.predict(X_test)

In [96]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

# calculating metrics for Gaussian Naive Bayes:
F1 = f1_score(Y_test, Y_pred)
accuracy = accuracy_score(Y_test, Y_pred)
AUC = roc_auc_score(Y_test, Y_pred)

print('Gaussian Naive Bayes metrics:')
print(f'F1 metrics: {F1:.2f}')
print(f'Accuracy metrics: {accuracy:.2f}')
print(f'Area under curve metrics: {AUC:.2f}')

Gaussian Naive Bayes metrics:
F1 metrics: 0.46
Accuracy metrics: 0.56
Area under curve metrics: 0.56


Выводы:
Проведена предобработка файла данных по кредитам.
Произведена балансировка выборки - то есть отбрасывание части сэмплов с open_account_flg == 0, чтобы выравнять количество кредитополучателей, получивших и не получивших кредит.
Произведено обучение моделей с помощью различных методов и выбраны методы, дающие наилучшие результаты согласно используемым метрикам.
